# MovieLens

In [2]:
import numpy as np
import pandas as pd

train_data = np.load('./RippleNet-PyTorch-baseline/data/train_data_original.npy', allow_pickle=True)
test_data = np.load('./RippleNet-PyTorch-baseline/data/test_data_original.npy', allow_pickle=True)
eval_data = np.load('./RippleNet-PyTorch-baseline/data/eval_data_original.npy', allow_pickle=True)
# 把train和eval合并成train，test单独test，过滤掉第三列是0的
# 合并train和eval数据
train_data = np.concatenate([train_data, eval_data])

# 过滤掉第三列(rating)为0的数据
train_data = train_data[train_data[:, 2] != 0]
test_data = test_data[test_data[:, 2] != 0]
# 变成dataframe
train_data = pd.DataFrame(train_data, columns=['user', 'item', 'interaction'])
test_data = pd.DataFrame(test_data, columns=['user', 'item', 'interaction'])
ratings = pd.read_csv('dataset/ripple/ratings.dat', sep='::', header=None, names=['userId', 'movieId', 'rating', 'timestamp'])
# 通过train data和test data，和ratingsmerge
train_data = pd.merge(train_data, ratings, left_on=['user', 'item'], right_on=['userId', 'movieId'], how='left')
train_data = train_data.drop(['user', 'item','interaction'], axis=1)
test_data = pd.merge(test_data, ratings, left_on=['user', 'item'], right_on=['userId', 'movieId'], how='left')
test_data = test_data.drop(['user', 'item','interaction'], axis=1)



/var/folders/q3/w058rs213y1fbnz4r36x5z2m0000gn/T/ipykernel_71918/509847103.py:17: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ratings = pd.read_csv('dataset/ripple/ratings.dat', sep='::', header=None, names=['userId', 'movieId', 'rating', 'timestamp'])


In [3]:
train_data

,userId,movieId,rating,timestamp
0,1,2355,5,978824291
1,1,2797,4,978302039
2,1,1097,4,978301953
3,1,2804,5,978300719
4,1,260,4,978300760
...,...,...,...,...
301850,3704,562,5,966284788
301851,4892,300,4,962735284
301852,2585,260,4,973876019
301853,2507,3949,5,988075138


In [4]:
train_data.groupby('userId').count().describe()

,movieId,rating,timestamp
count,6030.000000,6030.000000,6030.000000
mean,50.058872,50.058872,50.058872
std,54.247227,54.247227,54.247227
min,1.000000,1.000000,1.000000
25%,15.000000,15.000000,15.000000
50%,31.000000,31.000000,31.000000
75%,65.000000,65.000000,65.000000
max,729.000000,729.000000,729.000000


In [5]:
test_data.groupby('userId').count().describe()


,movieId,rating,timestamp
count,5876.000000,5876.000000,5876.000000
mean,12.765487,12.765487,12.765487
std,13.949214,13.949214,13.949214
min,1.000000,1.000000,1.000000
25%,4.000000,4.000000,4.000000
50%,8.000000,8.000000,8.000000
75%,17.000000,17.000000,17.000000
max,171.000000,171.000000,171.000000


In [6]:
train_data.to_csv('dataset/movielens_1M/train_set.csv', index=False)
test_data.to_csv('dataset/movielens_1M/test_set.csv', index=False)


In [8]:
import pickle
train_data = pd.concat([train_data, test_data])

# 保存用户ID列表
with open('dataset/movielens_1M/user_ids.pkl', 'wb') as f:
    pickle.dump(train_data['userId'].unique().tolist(), f)

# 保存电影ID列表 
with open('dataset/movielens_1M/movie_ids.pkl', 'wb') as f:
    pickle.dump(train_data['movieId'].unique().tolist(), f)

-----
DPO Dataset

In [15]:
import pandas as pd
pd.read_csv('dataset/movielens_1M/sequential_top3.csv')

,userId,history,candidates,groundtruth
0,1,"[1836, 3408, 260, 1961, 150, 1028, 1246, 3114,...","[2791, 209, 1025]","[2791, 3105]"
1,1,"[3186, 2804, 1207, 2028, 150, 1028, 1246, 3114...","[2791, 3564, 3802]","[2791, 3105]"
2,1,"[1836, 260, 2028, 150, 1028, 1246, 2762, 3114,...","[3875, 621, 2791]","[2791, 3105]"
3,1,"[3186, 3408, 2804, 1207, 260, 1028, 2797, 3114...","[2791, 337, 3847]","[2791, 3105]"
4,1,"[1836, 3408, 2804, 260, 2692, 1961, 1097, 1246...","[2791, 2417, 1915]","[2791, 3105]"
...,...,...,...,...
51405,6040,"[573, 1150, 2011, 1172, 3471, 1103, 3547, 2725...","[1080, 1749, 1183]","[1221, 1228, 1235, 3359, 1333, 1960, 1921, 326..."
51406,6040,"[1392, 1111, 1285, 2938, 1250, 80, 1354, 3547,...","[1221, 3105, 3188]","[1221, 1228, 1235, 3359, 1333, 1960, 1921, 326..."
51407,6040,"[649, 2076, 2301, 2011, 306, 899, 1206, 3547, ...","[1993, 1326, 720]","[1221, 1228, 1235, 3359, 1333, 1960, 1921, 326..."
51408,6040,"[1392, 3521, 1965, 2301, 3224, 908, 2938, 1136...","[891, 513, 1221]","[1221, 1228, 1235, 3359, 1333, 1960, 1921, 326..."


In [16]:
ratings = pd.read_csv('dataset/ripple/ratings.dat', sep='::', header=None, names=['userId', 'movieId', 'rating', 'timestamp'])
links = pd.read_csv('dataset/movielens/links.csv')
links.dropna(inplace=True)
movies_metadata = pd.read_csv('dataset/movielens/movies_metadata.csv')

/var/folders/q3/w058rs213y1fbnz4r36x5z2m0000gn/T/ipykernel_71918/3344706090.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ratings = pd.read_csv('dataset/ripple/ratings.dat', sep='::', header=None, names=['userId', 'movieId', 'rating', 'timestamp'])
/var/folders/q3/w058rs213y1fbnz4r36x5z2m0000gn/T/ipykernel_71918/3344706090.py:4: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_metadata = pd.read_csv('dataset/movielens/movies_metadata.csv')


link：

movieId(=movieId in ratings_small),

imdbId (imdb_id in movies_metadata), 

tmdbId (id in movies_metadata)

In [17]:
movies_metadata['id'] = movies_metadata['id'].astype('str')
links['tmdbId'] = links['tmdbId'].astype(int).astype('str')
movie = pd.merge(
    links[['movieId', 'tmdbId']],
    movies_metadata[['id', 'title', 'genres','original_language','popularity']],
    left_on='tmdbId',
    right_on='id',
    how='left'
)

In [19]:
def parse_genres(x):
    if pd.isna(x):
        return []
    try:
        if isinstance(x, str):
            return eval(x)
        return x
    except:
        return []
movie['genres'] = movie['genres'].apply(parse_genres)
movie['genres'] = movie['genres'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) and x and isinstance(x[0], dict) else x)


In [20]:
movie.head()

,movieId,tmdbId,id,title,genres,original_language,popularity
0,1,862,862,Toy Story,"[Animation, Comedy, Family]",en,21.946943
1,2,8844,8844,Jumanji,"[Adventure, Fantasy, Family]",en,17.015539
2,3,15602,15602,Grumpier Old Men,"[Romance, Comedy]",en,11.712900
3,4,31357,31357,Waiting to Exhale,"[Comedy, Drama, Romance]",en,3.859495
4,5,11862,11862,Father of the Bride Part II,[Comedy],en,8.387519


In [27]:
test_data.groupby('userId').count().describe()


,movieId,rating,timestamp
count,5876.000000,5876.000000,5876.000000
mean,12.765487,12.765487,12.765487
std,13.949214,13.949214,13.949214
min,1.000000,1.000000,1.000000
25%,4.000000,4.000000,4.000000
50%,8.000000,8.000000,8.000000
75%,17.000000,17.000000,17.000000
max,171.000000,171.000000,171.000000


In [47]:
seq_dataset = pd.read_csv('dataset/movielens_1M/sequential_top10.csv')
seq_dataset

,userId,history,candidates,groundtruth
0,1,"[1270, 1836, 3408, 2028, 150, 1028, 1246, 2762...","[1453, 3105, 1431, 2791, 198, 2568, 2507]","[2791, 3105]"
1,1,"[1836, 3408, 260, 2692, 1961, 1028, 1097, 2918...","[2530, 3105, 1379, 2511, 1063, 2791, 1272]","[2791, 3105]"
2,1,"[3186, 2804, 2692, 150, 1028, 2797, 2918, 3114...","[3563, 3105, 1058, 3478, 3533, 631, 2791, 356]","[2791, 3105]"
3,1,"[3186, 1836, 3408, 2797, 2762, 3114, 1545, 527...","[590, 1921, 745, 1280, 3210, 2312, 3105, 1005,...","[2791, 3105]"
4,1,"[3408, 2804, 260, 2692, 1961, 1028, 1097, 2797...","[3611, 2290, 1064, 3749, 3837, 2449, 823, 3105...","[2791, 3105]"
...,...,...,...,...
154225,6040,"[1096, 2070, 1250, 296, 80, 899, 1206, 1265, 3...","[2437, 3262, 3610, 1960, 3896, 673, 3032, 1333...","[1221, 1228, 1235, 3359, 1333, 1960, 1921, 326..."
154226,6040,"[1392, 3067, 908, 1172, 1354, 1265, 2395, 3751...","[1002, 746, 396, 1080, 70, 1235, 369, 1191, 39...","[1221, 1228, 1235, 3359, 1333, 1960, 1921, 326..."
154227,6040,"[1961, 527, 1288, 908, 1247, 2938, 1179, 3083,...","[1095, 514, 714, 2424, 3776, 3612, 3262, 1333,...","[1221, 1228, 1235, 3359, 1333, 1960, 1921, 326..."
154228,6040,"[1097, 2076, 3224, 1247, 899, 1103, 2303, 3547...","[926, 720, 1207, 126, 280, 1228, 1632, 647, 12...","[1221, 1228, 1235, 3359, 1333, 1960, 1921, 326..."


In [48]:
# 筛选掉groundtruth长度<=10的
seq_dataset = seq_dataset[seq_dataset['groundtruth'].apply(lambda x: len(eval(x)) >=10)]
seq_dataset = seq_dataset.reset_index(drop=True)

In [49]:
seq_dataset.to_csv('dataset/movielens_1M/sequential_top10_filtered.csv', index=False)

根据movie这个大表，和seq_dataset, 构建自然语言数据集

Natural Language dataset format: every row contains
1. sequential historical interactions (Movie ID + Movie Names + genre + spoken Language + popularDegree)
2. candidates (Movie ID + Movie Name + genre + spoken Language + popularDegree)
3. Groudtruth (Movie ID + Movie Name)


In [50]:
def get_movie_description(row):
    return f"Movie {row['movieId']}: {row['title']} (Genres: {', '.join(row['genres'])}, " \
           f"Language: {row['original_language']})"

def process_sequence(row):
    history = eval(row['history'])
    history_movies = movie[movie['movieId'].isin(history)].apply(get_movie_description, axis=1).tolist()
    
    candidates = eval(row['candidates'])
    candidate_movies = movie[movie['movieId'].isin(candidates)].apply(get_movie_description, axis=1).tolist()
    
    next_items = eval(row['groundtruth'])
    ground_truth = movie[movie['movieId'].isin(next_items)].apply(
        lambda x: f"Movie {x['movieId']}: {x['title']}", axis=1
    ).tolist()
    
    return {
        'history': history_movies,
        'candidates': candidate_movies,
        'ground_truth': ground_truth
    }

In [51]:
natural_language_dataset = seq_dataset[:10].apply(process_sequence, axis=1)


In [53]:
natural_language_dataset[0]


{'history': ['Movie 50: The Usual Suspects (Genres: Drama, Crime, Thriller, Language: en)',
  'Movie 176: Living in Oblivion (Genres: Drama, Comedy, Language: en)',
  'Movie 515: The Remains of the Day (Genres: Drama, Romance, Language: en)',
  'Movie 562: Welcome to the Dollhouse (Genres: Comedy, Drama, Language: en)',
  'Movie 1171: Bob Roberts (Genres: Comedy, Drama, Language: en)',
  'Movie 1392: Citizen Ruth (Genres: Drama, Comedy, Language: en)',
  'Movie 1732: The Big Lebowski (Genres: Comedy, Crime, Language: en)',
  'Movie 1759: Four Days in September (Genres: Action, Drama, Foreign, History, Thriller, Language: pt)',
  'Movie 1897: High Art (Genres: Drama, Romance, Language: en)',
  'Movie 3083: All About My Mother (Genres: Comedy, Drama, Language: es)'],
 'candidates': ['Movie 32: Twelve Monkeys (Genres: Science Fiction, Thriller, Mystery, Language: en)',
  'Movie 509: The Piano (Genres: Drama, Romance, Language: en)',
  'Movie 714: Dead Man (Genres: Drama, Fantasy, Western,

Build into DPO dataset